### Red Agents

Scenario 2 has two rules-based Red Agents. The first is our good friend the B_lineAgent. This represents an actor who has inside information, so is able to beeline straight towards the OpServer.

In [1]:
import random
import inspect
from os.path import dirname
from pprint import pprint

from CybORG import CybORG
from CybORG.Agents import *
from CybORG.Simulator.Actions import *
from CybORG.Simulator.Scenarios import FileReaderScenarioGenerator

path = inspect.getfile(CybORG)
path = dirname(path) + f'/Simulator/Scenarios/scenario_files/Scenario1b.yaml'
sg = FileReaderScenarioGenerator(path)

env = CybORG(scenario_generator=sg)

agent = B_lineAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(16):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    
    print(action)

DiscoverRemoteSystems 10.0.42.48/28
DiscoverNetworkServices 10.0.42.49
ExploitRemoteService 10.0.42.49
PrivilegeEscalate User1
DiscoverNetworkServices 10.0.128.107
ExploitRemoteService 10.0.128.107
PrivilegeEscalate Enterprise1
DiscoverRemoteSystems 10.0.128.96/28
DiscoverNetworkServices 10.0.128.97
ExploitRemoteService 10.0.128.97
PrivilegeEscalate Enterprise2
DiscoverNetworkServices 10.0.213.43
ExploitRemoteService 10.0.213.43
PrivilegeEscalate Op_Server0
Impact Op_Server0
Impact Op_Server0


/home/david/anaconda3/envs/ai/lib/python3.9/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


This agent runs along a predetermined path to the Op_Server, but is smart enough able to recover its position if interrupted. We can see below after Blue Team restores some hosts, the agent works out where the error in and re-exploits its way to the Op_Server.

In [2]:
action = Restore(hostname='Op_Server0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise2',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise1',session=0,agent='Blue')
env.step(action=action,agent='Blue')

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
            
    print(action)
    print('Success:',obs['success'])

Impact Op_Server0
Success: FALSE
PrivilegeEscalate Op_Server0
Success: FALSE
ExploitRemoteService 10.0.213.43
Success: FALSE
ExploitRemoteService 10.0.128.97
Success: TRUE
PrivilegeEscalate Enterprise2
Success: TRUE
DiscoverNetworkServices 10.0.213.43
Success: TRUE
ExploitRemoteService 10.0.213.43
Success: TRUE
PrivilegeEscalate Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE


The other red agent is the MeanderAgent. This performs a breadth first search on all known hosts, scanning each one in turn, before attempting a mix of exploit and privilege escalate on the rest. This is an extremely slow agent in contrast to the laser-focussed B_lineAgent.

In [3]:
agent = RedMeanderAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(46):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    
    print(action)

DiscoverRemoteSystems 10.0.44.64/28
DiscoverNetworkServices 10.0.44.76
DiscoverNetworkServices 10.0.44.73
DiscoverNetworkServices 10.0.44.72
DiscoverNetworkServices 10.0.44.69
DiscoverNetworkServices 10.0.44.70
PrivilegeEscalate User0
ExploitRemoteService 10.0.44.76
PrivilegeEscalate User4
DiscoverNetworkServices 10.0.162.225
ExploitRemoteService 10.0.44.69
PrivilegeEscalate User2
DiscoverNetworkServices 10.0.162.234
ExploitRemoteService 10.0.44.70
PrivilegeEscalate User1
ExploitRemoteService 10.0.44.73
ExploitRemoteService 10.0.44.72
PrivilegeEscalate User3
ExploitRemoteService 10.0.162.234
PrivilegeEscalate Enterprise1
DiscoverRemoteSystems 10.0.162.224/28
DiscoverNetworkServices 10.0.162.226
DiscoverNetworkServices 10.0.162.235
ExploitRemoteService 10.0.162.226
PrivilegeEscalate Enterprise2
DiscoverNetworkServices 10.0.248.233
ExploitRemoteService 10.0.162.225
PrivilegeEscalate Enterprise0
ExploitRemoteService 10.0.248.233
PrivilegeEscalate Op_Server0
Impact Op_Server0
Impact Op_Ser

The Meander Agent is also able to recover from Blue's disruption.

In [4]:
action = Restore(hostname='Op_Server0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise2',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise1',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

for i in range(24):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    print(env.get_last_action('Red'))

Impact Op_Server0
DiscoverRemoteSystems 10.0.248.224/28
ExploitRemoteService 10.0.162.235
PrivilegeEscalate Enterprise2
PrivilegeEscalate User1
PrivilegeEscalate User4
PrivilegeEscalate User0
PrivilegeEscalate Enterprise0
PrivilegeEscalate User2
PrivilegeEscalate Enterprise1
PrivilegeEscalate User3
ExploitRemoteService 10.0.162.226
PrivilegeEscalate User0
PrivilegeEscalate User1
PrivilegeEscalate Enterprise2
PrivilegeEscalate User2
PrivilegeEscalate User4
PrivilegeEscalate User3
ExploitRemoteService 10.0.248.233
PrivilegeEscalate User2
PrivilegeEscalate User4
PrivilegeEscalate User1
PrivilegeEscalate Op_Server0
PrivilegeEscalate User3


### Blue Agents

The BlueReactRemoveAgent will wait until it sees suspicious activity, before using remove on all the hosts it has flagged. However, due to the 5% change that Red's exploit is missed, Red will always eventually get to the Op_Server.

In [5]:
env = CybORG(sg,agents={'Red':B_lineAgent()})

agent = BlueReactRemoveAgent()

results = env.reset('Blue')
obs = results.observation
action_space = results.action_space

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    print(env.get_last_action('Blue'))

Monitor
Monitor
Monitor
Monitor
Monitor
Remove User2
Monitor
Monitor
Monitor
Monitor


The BlueReactRestoreAgent is the same as the React agent above, but uses the Restore action.

In [6]:
agent = BlueReactRestoreAgent()

results = env.reset('Blue')
obs = results.observation
action_space = results.action_space

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    print(env.get_last_action('Blue'))

Monitor
Monitor
Monitor
Restore User4
Monitor
Restore User4
Monitor
Restore User4
Monitor
Restore User4
Monitor
Restore User4


### Green Agent

An important part of CybORG Scenario1b is the Green agent, which represents the users on the network. The Green Agent is very simple, it only performs a scanning action on random hosts some of the time. This is only visible by Blue Agent.

In [7]:
agent = GreenAgent()

results = env.reset('Green')
obs = results.observation
action_space = results.action_space

for i in range(12):
    print(agent.get_action(obs,action_space))

Sleep
GreenPortScan
Sleep
Sleep
Sleep
GreenPortScan
GreenPortScan
GreenPortScan
Sleep
Sleep
GreenPortScan
Sleep


### Keyboard Agent

The KeyboardAgent allows a human user to manually choose actions. This is useful for getting an intuition for the scenario.

In [11]:
from CybORG.Agents.Wrappers import RedTableWrapper

cyborg = CybORG(sg,agents={'Blue':MonitorAgent()})
env = RedTableWrapper(env=cyborg, output_mode='table')

agent = KeyboardAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(3):
    print(obs)
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation

+----------------+-------------+----------+---------+------------+
|     Subnet     |  IP Address | Hostname | Scanned |   Access   |
+----------------+-------------+----------+---------+------------+
| 10.0.124.64/28 | 10.0.124.78 |  User0   |  False  | Privileged |
+----------------+-------------+----------+---------+------------+

************************************ Turn 1: Observation *************************************

+----------------+-------------+----------+---------+------------+
|     Subnet     |  IP Address | Hostname | Scanned |   Access   |
+----------------+-------------+----------+---------+------------+
| 10.0.124.64/28 | 10.0.124.78 |  User0   |  False  | Privileged |
+----------------+-------------+----------+---------+------------+

********************************* Turn 1: Command Selection **********************************

0 Sleep
1 DiscoverRemoteSystems
2 DiscoverNetworkServices
3 ExploitRemoteService
4 BlueKeep
5 EternalBlue
6 FTPDirectoryTraversal
7 Hara

----------------------------------------------------------------------------------------------
CHOOSE A COMMAND:  1


You chose DiscoverRemoteSystems.


******************************** Turn 1: Parameter Selection *********************************


-------------------------------------- Subnet Selection --------------------------------------
Automatically choosing 10.0.124.64/28 as it is the only option.
------------------------------------- Session Selection --------------------------------------
Automatically choosing 0 as it is the only option.
-------------------------------------- Agent Selection ---------------------------------------
Automatically choosing Red as it is the only option.
+----------------+-------------+-----------------+---------+------------+
|     Subnet     |  IP Address |     Hostname    | Scanned |   Access   |
+----------------+-------------+-----------------+---------+------------+
| 10.0.124.64/28 | 10.0.124.65 | UNKNOWN_HOST: 0 |  False  |    None    |
| 10.0.124.64/28 | 10.0.124.66 | UNKNOWN_HOST: 3 |  False  |    None    |
| 10.0.124.64/28 | 10.0.124.70 | UNKNOWN_HOST

----------------------------------------------------------------------------------------------
CHOOSE A COMMAND:  2


You chose DiscoverNetworkServices.


******************************** Turn 2: Parameter Selection *********************************


------------------------------------- Session Selection --------------------------------------
Automatically choosing 0 as it is the only option.
-------------------------------------- Agent Selection ---------------------------------------
Automatically choosing Red as it is the only option.
------------------------------------ Ip_address Selection ------------------------------------
0 10.0.124.78
1 10.0.124.65
2 10.0.124.74
3 10.0.124.70
4 10.0.124.66


...

You chose 10.0.124.78.
+----------------+-------------+-----------------+---------+------------+
|     Subnet     |  IP Address |     Hostname    | Scanned |   Access   |
+----------------+-------------+-----------------+---------+------------+
| 10.0.124.64/28 | 10.0.124.65 | UNKNOWN_HOST: 0 |  False  |    None    |
| 10.0.124.64/28 | 10.0.124.66 | UNKNOWN_HOST: 3 |  False  |    None    |
| 10.0.124.64/28 | 10.0.124.70 | UNKNOWN_HOST: 2 |  False  |    None    |
| 10.0.124.64/28 | 10.0.124.74 | UNKNOWN_HOST: 1 |  False  |    None    |
| 10.0.124.64/28 | 10.0.124.78 |      User0      |   True  | Privileged |
+----------------+-------------+-----------------+---------+------------+

************************************ Turn 3: Observation *************************************

+----------------+-------------+-----------------+---------+------------+
|     Subnet     |  IP Address |     Hostname    | Scanned |   Access   |
+----------------+-------------+-----------------+---------+------

----------------------------------------------------------------------------------------------
CHOOSE A COMMAND:  2


You chose DiscoverNetworkServices.


******************************** Turn 3: Parameter Selection *********************************


------------------------------------ Ip_address Selection ------------------------------------
0 10.0.124.78
1 10.0.124.65
2 10.0.124.74
3 10.0.124.70
4 10.0.124.66


----------------------------------------------------------------------------------------------
CHOOSE A PARAMETER:  0


You chose 10.0.124.78.
------------------------------------- Session Selection --------------------------------------
Automatically choosing 0 as it is the only option.
-------------------------------------- Agent Selection ---------------------------------------
Automatically choosing Red as it is the only option.
